# Transformer 모듈

이번 장에서는 language modeling task에 대해서 배운다. 
<br>

일련의 단어를 따르는 단어가 주어졌을때 우도의 확률에 대한 task이다.

<br>

sequence token이 우선 embedding layer를 우선 통과하고 뒤이어 positional encoding layer를 통과한다.  <br>
인코더는 TransformerEncoding layers로 이루어짐
square attention mask는 self-attention때문에 요구된다. 이 mask는 encoder이전의 위치에만 참고 가능함. <br>

모델링의 경우 모든 토큰은 마스킹을 해야한다. 

In [42]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):
    def __init__(self, ntoken : int, d_model : int, nhead:int, d_hid : int, nlayers : int, dropout : float = 0.5):
      super().__init__()
      self.model_type = 'Transformer'
      self.pos_encoder = PositionalEncoding(d_model, dropout)
      encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
      self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
      self.encoder = nn.Embedding(ntoken, d_model)
      self.d_model = d_model
      self.decoder = nn.Linear(d_model, ntoken)
      self.init_weights()  

    def init_weights(self) -> None:
      initrange = 0.1
      self.encoder.weight.data.uniform_(-initrange, initrange) # 가중치 범위
      self.decoder.bias.data.zero_()
      self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src:Tensor, src_mask : Tensor) -> Tensor:
      # src : Tensor, shape [seq_len, batch_size]
      # src_mask : Tensor, shape [seq_len, seq_len]
      # output : [seq_len, batchsize, ntoken]

        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

def generate_square_subsequent_mask(sz : int) -> Tensor:
  # triangular matrix 생성, 대각선은 0"
  return torch.triu(torch.ones(sz, sz)*float('-inf'), diagonal=1)


# positional encoding은 토큰의 절대위치나 비교되는 위치를 넣는다. embedding과 차원은 같아 합칠수 있다.
# sine, cosine 함수를 사용


class PositionalEncoding(nn.Module):
  def __init__(self, d_model:int, dropout:float=0.1, max_len:int = 5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)
    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position*div_term)
    pe[:, 0, 1::2] = torch.cos(position*div_term)
    self.register_buffer('pe', pe)


    def forward(self, x:Tensor)->Tensor:
      # x : [seq_len, batch_size, embedding_Dim]

      x = x+self.pe[:x.size(0)] # potision 정보를 주입
      return self.dropout(X)




In [54]:
# 배치의 경우 모델은 각각의 컬럼을 독자적으로 학습한다. 그래서 전체적으로 학습 불가

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter : dataset.IterableDataset) -> Tensor:
  "tensor로 변경"
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t:t.numel()>0, data)))

# train_iter의 경우 전처리과정에서 소비됨, 다시 만들어줌
train_iter, val_iter, test_iter = WikiText2()
train_data=data_process(train_iter)
val_data=data_process(val_iter)
test_data=data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data:Tensor, bsz:int) -> Tensor:
  # bsz 만큼 시퀀스를 나누고 나머지 버림
  # data : shape [N]
  # bsz : batchsize
  # return Tensor of shape[N//bsz, bsz]

  seq_len = data.size(0) // bsz
  data = data[:seq_len * bsz]
  data = data.view(bsz, seq_len).t().contiguous()
  return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [55]:
# input
bptt = 35
def get_batch(source:Tensor, i:int) -> Tuple[Tensor, Tensor]:
  # source [full_seq_len, batch_size]
  # i : int
  # tuple(data, target)
  seq_len = min(bptt, len(source)-1-i)
  data=source[i:i+seq_len]
  target=source[i+1:i+1+seq_len].reshape(-1)
  return data, target

In [56]:
ntokens=len(vocab)
emsize=200
d_hid=200
nlayers=2
nhead=2
dropout=0.2
model=TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [57]:
# run model
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval() 
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [59]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()



NotImplementedError: ignored

In [50]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

AttributeError: ignored